In [1]:
import tensorflow as tf
import numpy as np
import datetime

In [23]:
x_in = np.ones((10,10,10), dtype=np.float32)
y_res = np.ones((10,10,10), dtype=np.float32) * 2.

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'tb_logs/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

# l1 = tf.keras.layers.dense(10)
# @tf.function
def get_model():
  inputs = tf.keras.Input(shape=(10,10))
  l1 = tf.keras.layers.Dense(20, name="deft")(inputs)
  l2 = tf.keras.layers.Dense(10, name="deft2")(l1)
  # model = tf.keras.Sequential([tf.keras.layers.Dense(10)])
  model = tf.keras.Model(inputs=inputs, outputs=[l1,l2])
  print(model.output)
  return model

model = get_model()

@tf.function
def model_call(model, x_in, step=0, logging=True):
  out = model(x_in)
  
  # if step%10==0:
  # tf.summary.record_if(step%10==0)
  with train_summary_writer.as_default():
    tf.summary.histogram("bias", model.weights[1], step=step)
    tf.summary.histogram("weights", model.weights[0], step=step)
  return out
  

optimizer = tf.keras.optimizers.Adam()

[<tf.Tensor 'deft/BiasAdd_5:0' shape=(None, 10, 20) dtype=float32>, <tf.Tensor 'deft2/BiasAdd_4:0' shape=(None, 10, 10) dtype=float32>]


In [20]:
# pre_out = model.get_layer("deft").output
print(np.shape(model(x_in)))
print(np.shape(model(x_in)[1]))
# print(pre_out)
  

(2, 10, 10)
(10, 10, 10)


In [2]:
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
# stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "graph_log"
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)


Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


In [3]:

tf.summary.trace_on(graph=True, profiler=True)

z = model(x_in)
# model.summary()

with train_summary_writer.as_default():
  tf.summary.trace_export(
    name="graphy", step=0, profiler_outdir="graph_log")

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


In [4]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[ 0.1427598 ,  0.32253963,  0.25989383, -0.12723207, -0.39099985,
          0.15093285, -0.38602543,  0.03753287, -0.27563912,  0.52634645],
        [ 0.27221787, -0.11022073, -0.0578137 ,  0.36451977, -0.0694446 ,
          0.17148101,  0.4809699 ,  0.13938856,  0.3683995 ,  0.24940085],
        [-0.02136523, -0.50684446,  0.5445821 ,  0.23141927, -0.04572684,
          0.19577676,  0.50263333,  0.26486278, -0.39916456,  0.3580792 ],
        [-0.03082722, -0.47430134, -0.15615073,  0.4638064 ,  0.2624929 ,
          0.02976608,  0.33108592,  0.4067949 , -0.43859383, -0.12334606],
        [ 0.24692887,  0.3156559 ,  0.11799604,  0.43247503,  0.3342631 ,
         -0.43349415, -0.00057238, -0.201558  , -0.47169495, -0.44278973],
        [-0.23644581,  0.53655946,  0.37622184,  0.37652713,  0.43331796,
         -0.41483885,  0.1137706 , -0.33391142,  0.2306574 , -0.02919185],
        [ 0.5287248 , -0.28593695,  0.

In [5]:
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

logging=True
for i in range(50):
  with tf.GradientTape() as tape:
    out = model_call(model, x_in, step=i, logging=logging)
    loss = tf.reduce_mean((out - y_res)**2, axis=[1,2])

  grads = tape.gradient(loss, model.trainable_weights)
  # print(grads)
  with train_summary_writer.as_default():
    tf.summary.scalar("loss2", np.mean(loss), step=i)

  optimizer.apply_gradients(zip(grads, model.trainable_weights))
  print(f"{np.mean(loss):.2f}")

TypeError: record_if() takes 1 positional argument but 2 were given